In [2]:
!ls

2407.04716v1.pdf  application.py  README.md	    static     test.py
2407.05247v1.pdf  LICENSE	  requirements.txt  templates  Untitled.ipynb


In [74]:
! pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.8 MB/s eta 0:00:000m eta 0:00:0136m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 13.3 MB/s eta 0:00:00m eta 0:00:010:01:01
You should consider upgrading via the '/home/elena/Desktop/github_repos/text_to_speach/venv/bin/python -m pip install --upgrade pip' command.


In [6]:
import fitz

In [7]:
from PIL import Image

In [9]:
pdf_file = fitz.open('2407.04716v1.pdf')

## Page Dict

In [35]:
import pymupdf

pdf = pymupdf.open('2407.04716v1.pdf')
page = pdf[0]

In [36]:
images_and_text = []
doc_order = []

# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.get_text("dict")["blocks"]
for b in blocks:  # iterate through the text blocks
    if "ext" in b:
        if b["ext"] == "jpeg":
            print("image goes here", b["bbox"])
            images_and_text.append({"image": "image", "bbox": b["bbox"]})
            doc_order.append([s["bbox"][1], s["bbox"][3], "image"])
        else:
            print(b)
    elif "lines" in b:
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                print(round(s["size"]), s["origin"], s["bbox"])
                print("--------------")
                print(s["text"])
                print(" ")
                d = {}
                d["size"] = round(s["size"])
                d["origin"] = s["origin"]
                d["bbox"] = s["bbox"]
                d["text"] = s["text"]
                doc_order.append([s["bbox"][1], s["bbox"][3], "text"])
                images_and_text.append(d)
        print(b["bbox"])
    else:
        print(b)
    print("===============")
    print(" ")
            

14 (96.83499908447266, 140.9420166015625) (96.83499908447266, 130.182373046875, 515.3134765625, 144.54290771484375)
--------------
Closed-loop geothermal systems: Modeling and predictions
 
(96.83499908447266, 130.182373046875, 515.3134765625, 144.54290771484375)
 
11 (174.9550018310547, 179.7459716796875) (174.9550018310547, 171.56414794921875, 437.01348876953125, 182.47325134277344)
--------------
An e-print of this paper is available on arXiv.
 
(174.9550018310547, 171.56414794921875, 437.01348876953125, 182.47325134277344)
 
11 (270.0350036621094, 196.84197998046875) (270.0350036621094, 188.66015625, 341.9587097167969, 199.5692596435547)
--------------
Authored by
 
14 (262.6210021972656, 218.32196044921875) (262.6210021972656, 207.56231689453125, 349.35809326171875, 221.90850830078125)
--------------
K. Adhikari
 
11 (139.99700927734375, 232.5689697265625) (139.99700927734375, 224.38714599609375, 472.0264892578125, 235.29624938964844)
--------------
Graduate Student, Department of

In [37]:
for a in doc_order:
    print(a)

[130.182373046875, 144.54290771484375, 'text']
[171.56414794921875, 182.47325134277344, 'text']
[188.66015625, 199.5692596435547, 'text']
[207.56231689453125, 221.90850830078125, 'text']
[224.38714599609375, 235.29624938964844, 'text']
[238.63311767578125, 249.54222106933594, 'text']
[257.5352783203125, 271.8814697265625, 'text']
[274.3591003417969, 285.2681884765625, 'text']
[293.26129150390625, 307.60748291015625, 'text']
[310.08612060546875, 320.9952087402344, 'text']
[324.3321228027344, 335.2412109375, 'text']
[338.5354919433594, 349.48822021484375, 'text']
[338.5791320800781, 349.48822021484375, 'text']
[338.5354919433594, 349.48822021484375, 'text']
[338.5791320800781, 349.48822021484375, 'text']
[352.7825012207031, 363.7352294921875, 'text']
[352.8261413574219, 363.7352294921875, 'text']
[538.9169311523438, 550.5853881835938, 'text']
[538.9169311523438, 550.5853881835938, 'text']
[538.9169311523438, 550.5853881835938, 'text']
[538.9169311523438, 550.5853881835938, 'text']
[538.9

## Annotations

In [38]:
pdf = pymupdf.open('2407.04716v1.pdf')
page_content = pdf.load_page(0)

In [39]:
for annot in page_content.annots():
    print(f"Annotation: {annot.info}")

In [40]:
def insert_in_doc_order(shape):
    i = 0
    not_found = True
    while i < len(doc_order) and not_found:
        print(i, doc_order[i][0],shape[0])
        if doc_order[i][0] > shape[1]:
            print(i, "inserted")
            doc_order.insert(i, [shape[0], shape[1], "shape goes here"])
            not_found = False
        i = i + 1

In [41]:
from pymupdf import Rect, Point

newdoc = pymupdf.open()
page = newdoc.new_page()
nshape = page.new_shape()

i = 0
for shape in page_content.get_drawings():
    #print("shape", shape["items"], shape["type"])
    for item in shape['items']:
        if i >= 0:
            if item[0] == 're':
                nshape.draw_rect(item[1])
                insert_in_doc_order([item[1][1], item[1][3]])
            elif item[0] == 'l':
                nshape.draw_line(item[1], item[2])
                insert_in_doc_order([item[1][1], item[2][1]])
            elif item[0] == 'c':
                nshape.draw_bezier(item[1], item[2], item[3], item[4])
                #print(shape)
                insert_in_doc_order([item[1][1], item[3][1]])
            else:
                print(item)
    i = i + 1

    #print(shape['type'], shape['fill_opacity'], shape['fill'],
    #      shape['type'], shape['stroke_opacity'], shape['color'], shape['width'], shape['lineCap'], 
    #      shape['lineJoin'], shape['closePath'], shape['dashes'], shape['layer'], 
    #      shape['seqno'], shape['fill'], shape['fill_opacity'], shape['even_odd'])
    #nshape.finish(width = shape['width'], fill=shape['fill'])
    #shape_details = {'fill': shape['fill']}
    nshape.finish(fill=shape['fill'])

nshape.commit(overlay=True)
newdoc.save("output.pdf")

for page in newdoc:
    pix = page.get_pixmap()
    pix.save("newdoc.png")

0 130.182373046875 108.86199188232422
0 inserted
0 108.86199188232422 112.2490234375
1 130.182373046875 112.2490234375
1 inserted
0 108.86199188232422 159.26300048828125
1 112.2490234375 159.26300048828125
2 130.182373046875 159.26300048828125
3 171.56414794921875 159.26300048828125
3 inserted
0 108.86199188232422 161.0570068359375
1 112.2490234375 161.0570068359375
2 130.182373046875 161.0570068359375
3 159.26300048828125 161.0570068359375
4 171.56414794921875 161.0570068359375
4 inserted
0 108.86199188232422 376.708740234375
1 112.2490234375 376.708740234375
2 130.182373046875 376.708740234375
3 159.26300048828125 376.708740234375
4 161.0570068359375 376.708740234375
5 171.56414794921875 376.708740234375
6 188.66015625 376.708740234375
7 207.56231689453125 376.708740234375
8 224.38714599609375 376.708740234375
9 238.63311767578125 376.708740234375
10 257.5352783203125 376.708740234375
11 274.3591003417969 376.708740234375
12 293.26129150390625 376.708740234375
13 310.08612060546875 3

In [42]:
for d in doc_order:
    print(d)

[108.86199188232422, 110.45599365234375, 'shape goes here']
[112.2490234375, 112.2490234375, 'shape goes here']
[130.182373046875, 144.54290771484375, 'text']
[159.26300048828125, 159.26300048828125, 'shape goes here']
[161.0570068359375, 162.6510009765625, 'shape goes here']
[171.56414794921875, 182.47325134277344, 'text']
[188.66015625, 199.5692596435547, 'text']
[207.56231689453125, 221.90850830078125, 'text']
[224.38714599609375, 235.29624938964844, 'text']
[238.63311767578125, 249.54222106933594, 'text']
[257.5352783203125, 271.8814697265625, 'text']
[274.3591003417969, 285.2681884765625, 'text']
[293.26129150390625, 307.60748291015625, 'text']
[310.08612060546875, 320.9952087402344, 'text']
[324.3321228027344, 335.2412109375, 'text']
[338.5354919433594, 349.48822021484375, 'text']
[338.5791320800781, 349.48822021484375, 'text']
[338.5354919433594, 349.48822021484375, 'text']
[338.5791320800781, 349.48822021484375, 'text']
[352.7825012207031, 363.7352294921875, 'text']
[352.826141

## Images

In [31]:
images = page.get_images()

In [33]:
for image in images:
    print(image)
    xref = image[0]
    base_image = doc.extract_image(xref)
    with open(image[7], 'wb') as f:
        f.write(base_image['image'])

(108, 0, 667, 973, 8, 'ICCBased', '', 'I1', 'DCTDecode')
(109, 121, 684, 991, 8, 'ICCBased', '', 'I3', 'DCTDecode')
